<a href="https://colab.research.google.com/github/lx-47/gitaccess/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install Groq pygithub javalang

In [5]:
from google.colab import userdata
github_token = userdata.get('github_token')
groq_api_key = userdata.get('groq_api_key')

In [7]:
from github import Github
import os
gh = Github(github_token)
repo = gh.get_repo(os.getenv("GITHUB_REPO", "lx-47/todo-spring"))
def list_all_files_in_a_dir(folder_name, ref):
  a = repo.get_contents(folder_name,ref = ref)
  for i in a :
    print(i.path)

content=""

def read_all_files_in_a_dir(folder_name, ref):
  global content
  for i in repo.get_contents(folder_name, ref = ref):
    if(i.type == "dir"):
      print(i.path)
      read_all_files_in_a_dir(i.path)
    if i.type == "file" and i.path.endswith(".java"):
      file_content = i.decoded_content.decode()
      content += f"### {i.path}\n\n{file_content}\n\n"

list_all_files_in_a_dir("", "main")
read_all_files_in_a_dir("", "main")
print(content)

HELP.md
README.md
mvnw
mvnw.cmd
pom.xml
src
target
src
src/main
src/main/java
src/main/java/com
src/main/java/com/example
src/main/java/com/example/todo
src/main/java/com/example/todo/Mapper
src/main/java/com/example/todo/controller
src/main/java/com/example/todo/dto
src/main/java/com/example/todo/entity
src/main/java/com/example/todo/exception
src/main/java/com/example/todo/repository
src/main/java/com/example/todo/service
src/main/resources
src/test
src/test/java
src/test/java/com
src/test/java/com/example
src/test/java/com/example/todo
target
target/classes
target/classes/com
target/classes/com/example
target/classes/com/example/todo
target/classes/com/example/todo/controller
### src/main/java/com/example/todo/Mapper/TodoMapper.java

package com.example.todo.Mapper;


import com.example.todo.dto.TodoResponse;
import com.example.todo.entity.TodoEntity;

public class TodoMapper {
    public static TodoResponse todoResponse(TodoEntity todoEntity){
        return new TodoResponse(
     

In [9]:
import javalang

def generate_ast_for_content(content):
    """Generate AST for all Java files stored in the content variable."""
    file_blocks = content.split("### ")
    ast_dict = {}

    for block in file_blocks[1:]:  # Skip the first empty block
        header, java_code = block.split("\n\n", 1)
        file_name = header.strip()

        try:
            tree = javalang.parse.parse(java_code)
            ast_dict[file_name] = tree
        except Exception as e:
            print(f"Error parsing AST for {file_name}: {e}")

    return ast_dict


In [10]:
def build_symbol_table(ast_dict):
    """Build a symbol table from parsed AST."""
    symbol_table = {}

    for file_name, tree in ast_dict.items():
        for path, node in tree:
            if isinstance(node, javalang.tree.ClassDeclaration):
                symbol_table[node.name] = {
                    "type": "class",
                    "file": file_name,
                    "methods": [],
                    "fields": []
                }
            elif isinstance(node, javalang.tree.MethodDeclaration):
                class_name = path[-2].name
                symbol_table[class_name]["methods"].append(node.name)
            elif isinstance(node, javalang.tree.FieldDeclaration):
                class_name = path[-2].name
                for var in node.declarators:
                    symbol_table[class_name]["fields"].append(var.name)

    return symbol_table


In [11]:
import networkx as nx

def build_dependency_graph(ast_dict):
    """Create a dependency graph for all Java files."""
    dependency_graph = nx.DiGraph()

    for file_name, tree in ast_dict.items():
        dependency_graph.add_node(file_name)

        for path, node in tree:
            if isinstance(node, javalang.tree.Import):
                dependency_graph.add_edge(file_name, node.path)

    return dependency_graph


In [12]:
def analyze_scope(ast_dict):
    """Analyze scope for classes, methods, and variables."""
    scope_table = {}

    for file_name, tree in ast_dict.items():
        scope_data = {}

        for path, node in tree:
            if isinstance(node, javalang.tree.ClassDeclaration):
                class_name = node.name
                scope_data[class_name] = {"methods": [], "fields": []}
            elif isinstance(node, javalang.tree.MethodDeclaration):
                scope_data[class_name]["methods"].append(node.name)
            elif isinstance(node, javalang.tree.FieldDeclaration):
                for var in node.declarators:
                    scope_data[class_name]["fields"].append(var.name)

        scope_table[file_name] = scope_data

    return scope_table


In [18]:
# Generate AST for all files
ast_dict = generate_ast_for_content(content)
for file_name, tree in ast_dict.items():
    print(f"AST for {file_name}:")
    print(tree)
    print("\n")
# Build Symbol Table
symbol_table = build_symbol_table(ast_dict)
for class_name, class_info in symbol_table.items():
    print(f"Class: {class_name}")
    print(f"Type: {class_info['type']}")
    print(f"File: {class_info['file']}")
# Build Dependency Graph
dependency_graph = build_dependency_graph(ast_dict)

# Analyze Scope
scope_table = analyze_scope(ast_dict)
print(scope_table)
for file_name, scope_data in scope_table.items():
    print(f"Scope for {file_name}:")
    for class_name, class_info in scope_data.items():
        print(f"Class: {class_name}")

AST for src/main/java/com/example/todo/Mapper/TodoMapper.java:
CompilationUnit(imports=[Import(path=com.example.todo.dto.TodoResponse, static=False, wildcard=False), Import(path=com.example.todo.entity.TodoEntity, static=False, wildcard=False)], package=PackageDeclaration(annotations=None, documentation=None, modifiers=None, name=com.example.todo.Mapper), types=[ClassDeclaration(annotations=[], body=[MethodDeclaration(annotations=[], body=[ReturnStatement(expression=ClassCreator(arguments=[MethodInvocation(arguments=[], member=getId, postfix_operators=[], prefix_operators=[], qualifier=todoEntity, selectors=[], type_arguments=None), MethodInvocation(arguments=[], member=getTitle, postfix_operators=[], prefix_operators=[], qualifier=todoEntity, selectors=[], type_arguments=None), MethodInvocation(arguments=[], member=getDescription, postfix_operators=[], prefix_operators=[], qualifier=todoEntity, selectors=[], type_arguments=None), MethodInvocation(arguments=[], member=isCompleted, post

In [106]:
from groq import Groq

def call_model_to_identify_files(feature_description, ast_dict, symbol_table, dependency_graph, scope_table):
    client = Groq(api_key=groq_api_key)

    prompt = f"""
    You are a expert Spring Boot developer who is tasked to list only files paths that are required to be changed within <path></path> with short description of changes to be made for the new feature {feature_description}.
    - Do not required for unnessary file paths
    - Do not generate codes
    ## Repository Overview:
    - Symbol Table: {symbol_table}
    - Dependency Graph: {dependency_graph}
    - Scope Analysis: {scope_table}
    """

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="mistral-saba-24b",
        temperature=0.2,
        max_tokens=1000
    )
    return chat_completion.choices[0].message.content


In [107]:
feature_description = "add a Task Prioritization feature"
feature_prompt = call_model_to_identify_files(feature_description, ast_dict, symbol_table, dependency_graph, scope_table)
print(feature_prompt)

To add a Task Prioritization feature to the existing Spring Boot application, you need to make changes primarily in the following files. Here are the paths and brief descriptions of the changes required:

1. **TodoController.java**
   - **Path**: `src/main/java/com/example/todo/controller/TodoController.java`
   - **Changes**:
     - Add a new method to handle prioritization logic, such as `prioritizeTodo`.

2. **TodoRequest.java**
   - **Path**: `src/main/java/com/example/todo/dto/TodoRequest.java`
   - **Changes**:
     - Add a new field to represent priority, such as `priority`, and corresponding getter and setter methods.

3. **TodoEntity.java**
   - **Path**: `src/main/java/com/example/todo/entity/TodoEntity.java`
   - **Changes**:
     - Add a new field to represent priority, such as `priority`, and corresponding getter and setter methods.

4. **TodoResponse.java**
   - **Path**: `src/main/java/com/example/todo/dto/TodoResponse.java`
   - **Changes**:
     - Add a new field to re

In [ ]:
import re

def extract_paths(content):
    """Extract content between <path> tags and store in a list."""
    path_pattern = re.findall(r"<path>(.*?)</path>", content, re.DOTALL)
    paths_list = [path.strip() for path in path_pattern]
    return paths_list

paths = extract_paths(feature_prompt)
print(paths)
